In [606]:
# Fundamentos de Ciência de Dados
## PPGI/UFRJ 2024.2
### Profs Sergio Serra e Jorge Zavaleta
### Aluno Ubirajara S. Santos

In [607]:
import prov
import os


In [608]:
OUTPUT_DIR = './dados/saidas'
os.makedirs(OUTPUT_DIR, exist_ok=True) 

In [609]:
# Fontes de Dados
data_sources = {
     "amostras_rochas_fluidos": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-amostras-de-rochas-e-fluidos/acervo-de-amostras/consolidacao-2023.zip",
        "type": "zip"},
     "setores_sirgas": {
        "url": "https://www.gov.br/anp/pt-br/assuntos/exploracao-e-producao-de-oleo-e-gas/estudos-geologicos-e-geofisicos/arquivos-classificacao-de-modelos-exploratorios/setores-sirgas.zip",
        "type": "zip"},
     "blocos_exploratorios": {
        "url": "https://gishub.anp.gov.br/geoserver/BD_ANP/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=BD_ANP%3ABLOCOS_EXPLORATORIOS_SIRGAS&maxFeatures=40000&outputFormat=SHAPE-ZIP",
        "type": "zip"},
     "campos_producao": {
        "url": "https://gishub.anp.gov.br/geoserver/BD_ANP/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=BD_ANP%3ACAMPOS_PRODUCAO_SIRGAS&maxFeatures=40000&outputFormat=SHAPE-ZIP",
        "type": "zip"},
     "reservas_nacionais_hc": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-estatisticos/arquivos-reservas-nacionais-de-petroleo-e-gas-natural/tabela-dados-bar-2023.xlsx",
        "type": "xlsx"},
     "pocos_perfurados_2023": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-acervo-de-dados-tecnicos/pocos-publicos-2023.csv",
        "type": "csv"},
     "tabela_levantamentos_geoquimica": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-acervo-de-dados-tecnicos/tabela-levantamentos-geoquimicos.csv",
        "type": "csv"},
     "levantamento_sismico_2023": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-acervo-de-dados-tecnicos/sismicos-publicos-2023.csv",
        "type": "csv"},
     "tabela_pocos_2024": {
        "url": "./dados/entradas/Tabela_pocos_2024_Novembro_24.csv",
        "type": "csv", "sep": ";" ,"encoding": "ANSI"},
     "tabela_dados_geoquimica": {
        "url": "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos-acervo-de-dados-tecnicos/tabela-dados-geoquimicos.csv",
        "type": "csv",
        "header": 1}
}

In [629]:
import sys, datetime
from prov.model import ProvDocument, Namespace
from prov.dot import prov_to_dot
from IPython.display import Image
import plotly
import graphviz
import unicodedata
from pathlib import Path
import platform
import importlib.metadata

def gerar_prov_outputs(doc_prov):
    entity = "EDA-PROV"
    output_file = f"{entity}.png"
    try:
        dot = prov_to_dot(doc_prov)
        # Write to PNG
        dot.write_png(output_file)
        print(f"Provenance graph generated successfully: {output_file}")
        
    except Exception as e:
        print(f"Error generating provenance graph: {e}")
        # Save the DOT file for debugging
        with open("debug.dot", "w") as f:
            f.write(dot.to_string())
        print("Saved DOT file for debugging as 'debug.dot'.")
   
    # Serialização do documento
    doc_prov.serialize(entity + ".xml", format='xml') 
    doc_prov.serialize(entity + ".ttl", format='rdf', rdf_format='ttl',encoding="utf-8")
    print("Provenance serialized as XML and TTL.")
    

def adding_namespaces(document_prov):
    # Adiciona namespaces ao documento de proveniência.
    document_prov.add_namespace('void', 'http://vocab.deri.ie/void#')
    document_prov.add_namespace('ufrj', 'https://www.ufrj.br')
    document_prov.add_namespace('schema', 'http://schema.org/')    # Dados estruturados Schema.org
    document_prov.add_namespace('prov', 'http://www.w3.org/ns/prov#')     # Padrões PROV
    document_prov.add_namespace('foaf', 'http://xmlns.com/foaf/0.1/')     # Agentes FOAF
    document_prov.add_namespace('ufrj-ppgi', 'http://www.ufrj.br/ppgi/')  # UFRJ PPGI
    document_prov.add_namespace('anp', 'https://www.gov.br/anp/pt-br')    # ANP - Agência Nacional do Petróleo, Gás Natural e Biocombustíveis
    document_prov.add_namespace('anp-dados_tec','https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/acervo-de-dados-tecnicos') # ANP - Acervo de Dados Técnicos 
    document_prov.add_namespace('petrobras','https://petrobras.com.br/')  # PETROBRAS
    document_prov.add_namespace('br','http://br.org/ns/')    # Organizações Brasileiras
    return document_prov


def escape_label(text):
    """
    Escapes special characters for Graphviz.
    Encodes text to ASCII with XML character references.
    """
    return text.encode("ascii", "xmlcharrefreplace").decode()

def get_installed_packages():
    #Retorna os pacotes instalados no ambiente com suas versões.
    try:
        return {pkg.metadata['Name']: pkg.version for pkg in importlib.metadata.distributions()}
    except ImportError:
        import pkg_resources
        return {dist.project_name: dist.version for dist in pkg_resources.working_set}

def get_system_info():
    #Retorna informações do sistema.
    return {
        "OS": platform.system(),
        "OS Version": platform.version(),
        "OS Release": platform.release(),
        "Python Version": sys.version,
        "Python Executable": sys.executable,
        "Current Working Directory": str(Path.cwd()),}

def add_system_and_package_provenance(doc_prov, generation_activity):
    #Adiciona informações do sistema e pacotes ao documento de proveniência
    # Criar atividade para rastrear informações de sistema e pacotes
    #activity_id = "ufrj:track_system_and_packages"
    #tracking_activity = doc_prov.activity(activity_id, datetime.datetime.now(), None, {"prov:label": escape_label("Track system and package provenance")})
    # Criar atividade para rastrear informações de sistema e pacotes
    activity_id = "ufrj:track_system_and_packages"
    tracking_activity = doc_prov.activity(
        activity_id,
        datetime.datetime.now(), None, {"prov:label": escape_label("Track system and package provenance")}
    )

    # Associar a atividade ao agente do notebook
    if "ag-eda-ipynb" in dict_agents:
        doc_prov.wasAssociatedWith(tracking_activity, dict_agents["ag-eda-ipynb"])

    # Adicionar informações do sistema como entidades
    system_info = get_system_info()
    for key, value in system_info.items():
        sanitized_key = key.replace(" ", "_")  # Substituir espaços por _
        sys_entity = doc_prov.entity(f"schema:{sanitized_key}", {"prov:value": value})
        doc_prov.wasGeneratedBy(sys_entity, tracking_activity)

    # Adicionar pacotes instalados como entidades
    installed_packages = get_installed_packages()
    for pkg, version in installed_packages.items():
        pkg_entity = doc_prov.entity(f"schema:{pkg}", {"prov:value": version})
        doc_prov.wasGeneratedBy(pkg_entity, tracking_activity)

    return doc_prov

def create_agents(document_prov):
    
    #creating agents
    dagnts={} #cria dic
    dagnts["ag-orgbr"] = document_prov.agent("br:orgBr", {"prov:type":"prov:Organization", "foaf:name":escape_label("Oraganizações Brasileiras")})
    dagnts["ag-anp"] = document_prov.agent("anp:ANP", {"prov:type":"prov:Organization", "foaf:name":escape_label("Agência Nacional do Petróleo, Gás Natural e Biocombustíveis")})
    dagnts["ag-ufrj"] = document_prov.agent("ufrj:UFRJ", {"prov:type":"prov:Organization", "foaf:name":escape_label("Universidade Federal do Rio de Janeiro")})
    dagnts["ag-ppgi"] = document_prov.agent("ufrj:PPGI", {"prov:type":"prov:Organization", "foaf:name":escape_label("Programa de Pós Graduação em Informática")})
    dagnts["ag-greco"] = document_prov.agent("ufrj:GRECO", {"prov:type":"prov:Organization", "foaf:name":escape_label("Grupo de Engenharia do Conhecimento")})
    dagnts["ag-author-ubirajara"] = document_prov.agent("ufrj:Ubirajara", {"prov:type":"prov:Person", "foaf:name":escape_label("Ubirajara Simões Santos"), "foaf:mbox":"ubirajas@hotmail.com"})
    dagnts["ag-author-sergio"] = document_prov.agent("ufrj:Sergio", {"prov:type":"prov:Person", "foaf:name":escape_label("Sergio Serra"), "foaf:mbox":"serra@ppgi.ufrj.br"})
    dagnts["ag-author-jorge"] = document_prov.agent("ufrj:Jorge", {"prov:type":"prov:Person", "foaf:name":escape_label("Jorge Zavaleta"), "foaf:mbox":"zavaleta@pet-si.ufrrj.br"})
    dagnts["ag-petrobras"] = document_prov.agent("petrobras:Petrobras", {"prov:type":"prov:Organization", "foaf:name":escape_label("Petróleo Brasiliero S.A")})
    dagnts["ag-eda-ipynb"] = document_prov.agent("ufrj:eda.ipynb", {"prov:type":"prov:SoftwareAgent", "foaf:name":escape_label("eda.ipynb"), "prov:label":escape_label("Notebook Python utilizado no trabalho")})
    return dagnts

def associate_ufrj_agents(agents_dictionary):
    agents_dictionary["ag-anp"].actedOnBehalfOf(agents_dictionary["ag-orgbr"])
    agents_dictionary["ag-petrobras"].actedOnBehalfOf(agents_dictionary["ag-orgbr"])
    agents_dictionary["ag-ufrj"].actedOnBehalfOf(agents_dictionary["ag-orgbr"])
    agents_dictionary["ag-ppgi"].actedOnBehalfOf(agents_dictionary["ag-ufrj"])
    agents_dictionary["ag-greco"].actedOnBehalfOf(agents_dictionary["ag-ppgi"])
    agents_dictionary["ag-author-ubirajara"].actedOnBehalfOf(agents_dictionary["ag-greco"])
    agents_dictionary["ag-author-ubirajara"].actedOnBehalfOf(agents_dictionary["ag-petrobras"])
    agents_dictionary["ag-author-sergio"].actedOnBehalfOf(agents_dictionary["ag-ppgi"])
    agents_dictionary["ag-author-jorge"].actedOnBehalfOf(agents_dictionary["ag-ppgi"])
    agents_dictionary["ag-eda-ipynb"].actedOnBehalfOf(agents_dictionary["ag-ppgi"])
    return agents_dictionary

def create_initial_activities(document_prov):
    #creating activities
    #dataDownloadDatasets = datetime.datetime.strptime('29/11/24', '%d/%m/%y')
    
    dativs={}
    dativs["act-create-ds"] = document_prov.activity("anp:create-dataset", None, None, {"prov:label":escape_label( "Criação de datasets pela ANP")})
    #dativs["act-extract-ds"] = document_prov.activity("ufrj:extract-dataset")
    dativs["act-create-ds-eda"] = document_prov.activity("ufrj:create-ds-eda", None, None, {"prov:label":escape_label( "Criação de datasets para EDA")})
    
    dativs["act-load-ds-eda"] = document_prov.activity("ufrj:load-ds-eda")
    dativs["act-save-ipynb"] = document_prov.activity("ufrj:save-ipynb", None, None, {"prov:label":escape_label("Salvar notebook EDA")})
    return dativs

def cria_entidades_iniciais(document_prov):
    #creating entidades
    dents={}
    
    # Entidade para amostras de rochas e fluidos
    dents["ent-amostras-rochas-fluidos"] = document_prov.entity('anp:amostras_rochas_fluidos', {'prov:label':escape_label('Dataset com amostras de rochas e fluidos'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Consolidado 2023 de amostras disponíveis.'), 'prov:format': 'zip' })
    # Entidade para setores SIRGAS
    dents["ent-setores-sirgas"] = document_prov.entity('anp:setores_sirgas', {'prov:label':escape_label('Setores SIRGAS'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Modelos exploratórios em formato SIRGAS.'), 'prov:format': 'zip'})
    # Entidade para blocos exploratórios
    dents["ent-blocos-exploratorios"] = document_prov.entity('anp:blocos_exploratorios', {'prov:label':escape_label( 'Blocos exploratórios'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Blocos exploratórios com dados geoespaciais.'), 'prov:format': 'zip'})
    # Entidade para campos de produção
    dents["ent-campos-producao"] = document_prov.entity('anp:campos_producao', {'prov:label':escape_label( 'Campos de Produção'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Dados dos campos de produção em formato SIRGAS.'), 'prov:format': 'zip'})
    # Entidade para reservas nacionais de hidrocarbonetos
    dents["ent-reservas-nacionais-hc"] = document_prov.entity('anp:reservas_nacionais_hc',{'prov:label':escape_label( 'Reservas Nacionais de Hidrocarbonetos'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Tabela com dados sobre reservas nacionais.'), 'prov:format': 'xlsx'})
    # Entidade para poços perfurados (2023)
    dents["ent-pocos-perfurados-2023"] = document_prov.entity('anp:pocos_perfurados_2023',{'prov:label':escape_label( 'Poços perfurados - 2023'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('CSV com os poços perfurados no ano de 2023.'), 'prov:format': 'csv'})
    # Entidade para tabela de levantamentos geoquímicos
    dents["ent-tabela-levantamentos-geoquimica"] = document_prov.entity('anp:tabela_levantamentos_geoquimica',{'prov:label':escape_label( 'Tabela de levantamentos geoquímicos 20/04/2022'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Dados sobre levantamentos geoquímicos.'), 'prov:format': 'csv'})
     # Entidade para tabela de dados geoquímicos
    dents["ent-tabela-dados-geoquimica"] = document_prov.entity('anp:tabela_dados_geoquimica',{'prov:label':escape_label( 'Tabela_dados_geoquimica 06/08/2021'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Dados geoquímicos.'), 'prov:format': 'csv'})
     # Entidade para levantamento sísmico (2023)
    dents["ent-levantamento-sismico-2023"] = document_prov.entity('anp:levantamento_sismico_2023', {'prov:label':escape_label( 'Levantamento Sísmico - 2023'), 'prov:type': 'void:Dataset', 'prov:description':escape_label('CSV com dados de levantamentos sísmicos públicos.'), 'prov:format': 'csv'})
    # Entidade para tabela de poços (2024)
    dents["ent-tabela-pocos-2024"] = document_prov.entity('anp:tabela_pocos_2024', {'prov:label':escape_label( 'Tabela de Poços - 2024'.encode("ascii", "xmlcharrefreplace").decode()), 'prov:type': 'void:Dataset', 'prov:description':escape_label('Tabela CSV com dados atualizados de poços para 2024.'), 'prov:format': 'csv'})
     # Entidade para ANP dados técnicos
    dents["ent-anp-dados_tec-ds"] = document_prov.entity('anp-dados_tec:dataset', {'prov:label':escape_label( 'ANP Dataset de Dados Técnicos'.encode("ascii", "xmlcharrefreplace").decode()),'prov:type': 'void:Dataset','prov:description':escape_label('Dataset com dados técnicos disponíveis publicamente.'),'prov:format': 'csv'})
    
    # Entidade script python
    dents["ent-eda-ipynb"] = document_prov.entity('ufrj:eda-ipyn', {'prov:label':escape_label( "Notebook Python utilizado no trabalho".encode("ascii", "xmlcharrefreplace").decode()), 'prov:type': 'foaf:Document'})
    # Entidade Git
    dents["ent-git-eda"] = document_prov.entity('anp:github-eda', {'prov:label':escape_label( 'Repositorio Eba da ANP'.encode("ascii", "xmlcharrefreplace").decode()), 'prov:type': 'prov:Collection'})
    return dents
  

def initial_association_agents_activities_entities(document_prov, dictionary_agents, dictionary_activities, dictionary_entities):
    
    #Associate activity of generate dataset with ANP agent
    document_prov.wasAssociatedWith(dictionary_activities["act-create-ds"], dictionary_agents["ag-anp"])
    
    #Associating datasets with activities of generate eba datasets
    document_prov.wasGeneratedBy(dictionary_entities["ent-amostras-rochas-fluidos"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-setores-sirgas"], dictionary_activities["act-create-ds"])    
    document_prov.wasGeneratedBy(dictionary_entities["ent-blocos-exploratorios"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-campos-producao"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-reservas-nacionais-hc"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-pocos-perfurados-2023"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-tabela-levantamentos-geoquimica"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-tabela-dados-geoquimica"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-levantamento-sismico-2023"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-tabela-pocos-2024"], dictionary_activities["act-create-ds"])
    document_prov.wasGeneratedBy(dictionary_entities["ent-anp-dados_tec-ds"], dictionary_activities["act-create-ds"])
    
    
    
    #Associating ZIPs, XLSX, CSV com entities do dataset genérico
    #document_prov.wasDerivedFrom(dictionary_entities["ent-dredfp2021-zip"], dictionary_entities["ent-dredfp"])  
       
    #associate activity of eda, com autor
    document_prov.wasAssociatedWith(dictionary_activities["act-create-ds-eda"], dictionary_agents["ag-author-ubirajara"])   

    #associate notebook agent with eba dataset
    document_prov.wasAssociatedWith(dictionary_activities["act-create-ds-eda"], dictionary_agents["ag-eda-ipynb"])    
             
    #associate eda github repository with store datasets activity
    document_prov.wasGeneratedBy(dictionary_entities["ent-git-eda"], dictionary_activities["act-save-ipynb"])
    
    
def initProvenance():
    #Inicializa o documento de proveniência com namespaces, agentes, atividades e entidades.
    
    global doc_prov, dict_agents, dict_activities, dict_entities

    # Criando um documento vazio de proveniência
    doc_prov = ProvDocument()

    # Criar namespaces no documento de proveniência
    doc_prov = adding_namespaces(doc_prov)

    # Criar agentes
    dict_agents = create_agents(doc_prov)

    # Criar atividades iniciais
    dict_activities = create_initial_activities(doc_prov)

    # Criar entidades iniciais
    dict_entities = cria_entidades_iniciais(doc_prov)

    # Criar hierarquia de agentes
    dict_agents = associate_ufrj_agents(dict_agents)

    # Associar agentes, atividades e entidades
    initial_association_agents_activities_entities(doc_prov, dict_agents, dict_activities, dict_entities)

    # Adicionar proveniência do sistema e pacotes
    doc_prov = add_system_and_package_provenance(doc_prov, dict_agents)

    return doc_prov, dict_agents, dict_activities, dict_entities


    


In [630]:
import pandas as pd
import matplotlib
import openpyxl
from io import BytesIO
import requests
import zipfile
import datetime


In [631]:
def extract_csv_from_zip(url, temp_dir="./dados/temp"):
    """
    Baixa, extrai e retorna os caminhos dos arquivos CSV contidos em um ZIP.
    """
    os.makedirs(temp_dir, exist_ok=True)
    csv_files = []

    try:
        # Baixar o arquivo ZIP
        response = requests.get(url)
        if response.status_code != 200:
            raise ValueError(f"Erro ao baixar o arquivo: {response.status_code}")
        
        # Abrir o ZIP em memória e extrair
        with zipfile.ZipFile(BytesIO(response.content)) as zf:
            zf.extractall(temp_dir)
            extracted_files = zf.namelist()

        # Identificar e coletar arquivos CSV
        for file in extracted_files:
            if file.endswith(".csv"):
                file_path = os.path.join(temp_dir, file)
                csv_files.append({"name": os.path.basename(file), "path": file_path})

        if not csv_files:
            raise ValueError("Nenhum arquivo CSV encontrado no ZIP.")
        
        print(f"Arquivos CSV extraídos: {[file['name'] for file in csv_files]}")
        return csv_files

    except Exception as e:
        print(f"Erro ao processar ZIP: {e}")
        return None
        
def update_data_sources_with_zip(data_sources, source_name, temp_dir="./temp"):
    """
    Atualiza `data_sources` com os arquivos CSV extraídos de um ZIP.
    """
    source = data_sources.get(source_name)
    if not source or source.get("type") != "zip":
        print(f"Fonte '{source_name}' não encontrada ou não é um ZIP.")
        return data_sources

    url = source["url"]
    csv_files = extract_csv_from_zip(url, temp_dir=temp_dir)

    if csv_files:
        for i, csv_file in enumerate(csv_files):
            new_source_name = f"{source_name}_csv_{i+1}"
            data_sources[new_source_name] = {
                "url": csv_file,
                "type": "csv",
                "sep": source.get("sep", ";"),
                "encoding": source.get("encoding", "utf-8"),
                "header": source.get("header", 0),
                "date_columns": source.get("date_columns", [])
            }
        print(f"Data sources atualizados com os arquivos CSV de '{source_name}'.")
    return data_sources


In [632]:
def load_data_from_source_csv(source_name, data_sources):
    """
    Carrega dados com base no nome da fonte e na configuração em data_sources.
    Returns:
        pd.DataFrame: DataFrame com os dados carregados, ou None se houver erro.
    """
    global doc_prov, dict_agents,  dict_activities, dict_entities  # Declare global variables
    #save execution start time
    execStartTime = datetime.datetime.now()

    source = data_sources.get(source_name)
   
    if not source:
        print(f"Fonte '{source_name}' não encontrada em data_sources.")
        return None

    file_type = source.get("type")
    url = source.get("url")
    sep = source.get("sep", ";")  # Valor padrão para CSV
    encoding = source.get("encoding", "utf-8")  # Valor padrão para codificação
    date_columns = source.get("date_columns", [])  

    try:
        if file_type == "csv":
             # Caso específico para tabela_pocos_2024
            if source_name == "tabela_pocos_2024":
                df = pd.read_csv(url, encoding="ANSI", sep=sep)
            # Caso específico para tabela_dados_geoquimica
            elif source_name == "tabela_dados_geoquimica":
                df = pd.read_csv(url, sep=sep, encoding=encoding, header=1)  # Cabeçalho na segunda linha
            else:
                df = pd.read_csv(url, sep=sep, encoding=encoding, parse_dates=date_columns)
        elif file_type == "xlsx":
            df = pd.read_excel(url)
        else:
            print(f"Tipo de arquivo '{file_type}' não suportado.")
            return None
        print(f"Dados carregados com sucesso para '{source_name}'.")
    
        # End execution time for provenance tracking
        execEndTime = datetime.datetime.now()
    
        # Criar atividade com horário de término da execução
        activity_key = f"act-carga-{source_name}"
        dict_activities[activity_key] = doc_prov.activity(f"ufrj:carga_{source_name}", execStartTime, execEndTime)
    
        # Associar a atividade ao agente
        doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])
        
        # Associar a atividade com os dados carregados
        entity_key = f"ent-{source_name}"
        dict_entities[entity_key] = doc_prov.entity(f"ufrj:{source_name}", {
            "prov:label": escape_label(f"Dataset carregado: {source_name}"),"prov:type": "void:Dataset", "prov:generatedAtTime": execEndTime.isoformat(),})
        doc_prov.wasGeneratedBy(dict_entities[entity_key], dict_activities[activity_key])
    
        # Associar a atividade ufrj:carga à entidade correspondente criada pela ANP
        anp_entity_key = f"ent-{source_name.replace('_', '-')}"  # Convert to ANP format (e.g., `tabela_pocos_2024` -> `ent-tabela-pocos-2024`)
        if anp_entity_key in dict_entities:
            # Establish the prov:used relationship
            doc_prov.used(dict_activities[activity_key], dict_entities[anp_entity_key])
        else:
            print(f"Warning: ANP entity '{anp_entity_key}' not found for activity '{activity_key}'.")
        return df
        
    except Exception as e:
        print(f"Erro ao carregar dados de '{source_name}': {e}")
        return None

In [633]:
#df_pocos_orig = load_data_from_source("tabela_pocos_2024", data_sources)

In [634]:
#df_pocos_orig.columns


In [635]:
#df_sismica_2023_orig = load_data_from_source("levantamento_sismico_2023", data_sources)

In [636]:
#df_sismica_2023_orig.columns

In [637]:
#df_lev_geoq_2022 = load_data_from_source("tabela_levantamentos_geoquimica", data_sources)

In [638]:
#df_lev_geoq_2022.columns

In [639]:
#df_geoq_2021 = load_data_from_source("tabela_dados_geoquimica", data_sources)

In [640]:
#df_geoq_2021.columns

In [641]:
#df_reservas = load_data_from_source("reservas_nacionais_hc", data_sources)

In [642]:
#df_reservas.columns

In [643]:
##df_amostras = load_data_from_source("amostras_rochas_fluidos", data_sources)

In [644]:
##data_sources = update_data_sources_with_zip(data_sources, "amostras_rochas_fluidos")

In [645]:
"""def create_eda_dataset():
    global doc_prov, dict_agents, dict_activities, dict_entities

    # Load datasets and associate with provenance
    df_sismica_2023_orig = load_data_from_source_csv("levantamento_sismico_2023", data_sources)
    df_pocos_orig = load_data_from_source_csv("tabela_pocos_2024", data_sources)
    df_lev_geoq_2022 = load_data_from_source_csv("tabela_levantamentos_geoquimica", data_sources)
    df_geoq_2021 = load_data_from_source_csv("tabela_dados_geoquimica", data_sources)
    df_reservas = load_data_from_source_csv("reservas_nacionais_hc", data_sources)

    # Debugging provenance dictionaries
    print("Activities after data load:", dict_activities.keys())
    print("Entities after data load:", dict_entities.keys())"""
    

def create_eda_dataset():
    """
    Lógica para criar o dataset EDA.
    Adicione toda a lógica para manipulação de dados e associação de proveniência.
    """
    activity_key = "act-create-ds-eda"
    exec_start = datetime.datetime.now()
    global doc_prov, dict_agents, dict_activities, dict_entities

    # Load datasets and associate with provenance
    df_sismica_2023_orig = load_data_from_source_csv("levantamento_sismico_2023", data_sources)
    df_pocos_orig = load_data_from_source_csv("tabela_pocos_2024", data_sources)
    df_lev_geoq_2022 = load_data_from_source_csv("tabela_levantamentos_geoquimica", data_sources)
    df_geoq_2021 = load_data_from_source_csv("tabela_dados_geoquimica", data_sources)
    df_reservas = load_data_from_source_csv("reservas_nacionais_hc", data_sources)

    # Debugging provenance dictionaries
    print("Activities after data load:", dict_activities.keys())
    print("Entities after data load:", dict_entities.keys())
    
    exec_end = datetime.datetime.now()

    # Atualiza atividade no documento de proveniência
    dict_activities[activity_key] = doc_prov.activity(
        "ufrj:create-ds-eda", exec_start, exec_end,
        {"prov:label": escape_label("Criação de datasets para EDA")}
    )
    doc_prov.wasAssociatedWith(dict_activities[activity_key], dict_agents["ag-eda-ipynb"])

    print("Dataset creation tracked in provenance.")




In [646]:
def main():
    """
    Função principal para inicializar e executar o rastreamento de proveniência.
    """
    global doc_prov, dict_agents, dict_activities, dict_entities

    # Inicializar o documento de proveniência
    doc_prov, dict_agents, dict_activities, dict_entities = initProvenance()

    # Verificar a inicialização
    print("Agents:", dict_agents.keys())
    print("Activities:", dict_activities.keys())
    print("Entities:", dict_entities.keys())

    # Executar a lógica de criação do dataset
    create_eda_dataset()

    # Serializar e gerar saídas
    gerar_prov_outputs(doc_prov)

In [627]:
"""def main():
    # Inicializa o documento de proveniência, agentes, atividades e entidades
    print("Initializing Provenance Tracking...")
    global doc_prov, dict_agents, dict_activities, dict_entities
    doc_prov, dict_agents, dict_activities, dict_entities = initProvenance()

    # Verifica a inicialização
    print("Agents:", dict_agents.keys())
    print("Activities:", dict_activities.keys())
    print("Entities:", dict_entities.keys())

    # Carrega datasets e associa suas proveniências
    print("Loading datasets and tracking provenance...")
    datasets = {
        "levantamento_sismico_2023": {"type": "csv", "url": "path_to_file.csv", "sep": ";"},
        "tabela_pocos_2024": {"type": "csv", "url": "path_to_file.csv", "sep": ";"},
        # Adicione outros datasets conforme necessário
    }

    for source_name, config in datasets.items():
        load_data_from_source_csv(source_name, datasets)

    # Executa a lógica de criação do dataset para EDA
    print("Creating EDA dataset...")
    create_eda_dataset()

    # Serializa e gera saídas
    print("Generating provenance outputs...")
    gerar_prov_outputs(doc_prov)

    print("Provenance Tracking Completed.")"""


In [647]:
#EXECUTAR

main()

Agents: dict_keys(['ag-orgbr', 'ag-anp', 'ag-ufrj', 'ag-ppgi', 'ag-greco', 'ag-author-ubirajara', 'ag-author-sergio', 'ag-author-jorge', 'ag-petrobras', 'ag-eda-ipynb'])
Activities: dict_keys(['act-create-ds', 'act-create-ds-eda', 'act-load-ds-eda', 'act-save-ipynb'])
Entities: dict_keys(['ent-amostras-rochas-fluidos', 'ent-setores-sirgas', 'ent-blocos-exploratorios', 'ent-campos-producao', 'ent-reservas-nacionais-hc', 'ent-pocos-perfurados-2023', 'ent-tabela-levantamentos-geoquimica', 'ent-tabela-dados-geoquimica', 'ent-levantamento-sismico-2023', 'ent-tabela-pocos-2024', 'ent-anp-dados_tec-ds', 'ent-eda-ipynb', 'ent-git-eda'])
Dados carregados com sucesso para 'levantamento_sismico_2023'.
Dados carregados com sucesso para 'tabela_pocos_2024'.
Dados carregados com sucesso para 'tabela_levantamentos_geoquimica'.
Dados carregados com sucesso para 'tabela_dados_geoquimica'.
Dados carregados com sucesso para 'reservas_nacionais_hc'.
Activities after data load: dict_keys(['act-create-ds', 

In [553]:
def main():
    #Função principal para gerar o documento de proveniência.
    
    # Initialize provenance objects
    global doc_prov, dict_agents, dict_activities, dict_entities

    # Adicionar inicialização de proveniência, agentes e atividades
    doc_prov, dict_agents, dict_activities, dict_entities = initProvenance()


    # Check initializationAssociatedWith(tracking_activ
    print("Agents:", dict_agents.keys())
    print("Activities:", dict_activities.keys())
    print("Entities:", dict_entities.keys())

    # Executa a lógica de criação do dataset
    create_eda_dataset()

    # Serializa e gera saídas
    gerar_prov_outputs(doc_prov)

    
main()

NameError: name 'tracking_activity' is not defined